# ***YOLOv12 + MobileNetV3-small + PANet***

In [2]:
# Install required packages (for Google Colab)
!pip install -q ultralytics roboflow

import os
from pathlib import Path
import yaml
from roboflow import Roboflow
from ultralytics import YOLO

# Konfigurasi Roboflow
API_KEY = "QOd5ldAdjiaehHn5m6WC"
WORKSPACE = "dentalogic8"
PROJECT_ID = "dental-caries-7kttb"
VERSION_NUM = 10

# Download Dataset
rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT_ID)
dataset = project.version(VERSION_NUM).download("yolov12")

DATASET_DIR = Path(f"/content/dental-caries-{VERSION_NUM}")
DATA_YAML = DATASET_DIR / "data.yaml"

# Verifikasi download
if DATA_YAML.exists():
    print(f"✅ Dataset berhasil diunduh: {DATA_YAML}")
    print(f"Struktur dataset: {os.listdir(DATASET_DIR)}")
else:
    raise FileNotFoundError("Dataset gagal diunduh. Periksa koneksi dan kredensial Roboflow.")

# Custom YAML untuk YOLOv12 dengan MobileNetV3-Small backbone via TorchVision
custom_yaml_content = """
nc: 7  # number of classes
scales:  # model compound scaling constants
  n: [0.50, 0.25, 1024]
  s: [0.50, 0.50, 1024]
  m: [0.50, 1.00, 512]
  l: [1.00, 1.00, 512]
  x: [1.00, 1.50, 512]

# MobileNetV3-Small backbone via TorchVision (adapted to match original YOLOv12 outputs)
backbone:
  # [from, repeats, module, args]
  - [-1, 1, TorchVision, [0, mobilenet_v3_small, DEFAULT, True, 2, True]]  # 0: MobileNetV3-Small
  - [0, 1, Index, [24, 3]]  # 1: P3 raw (24ch, /8)
  - [1, 1, Conv, [256, 1, 1]]  # 2: P3 projected (256ch, /8)
  - [0, 1, Index, [48, 8]]  # 3: P4 raw (48ch, /16)
  - [3, 1, Conv, [512, 1, 1]]  # 4: P4 projected (512ch, /16)
  - [0, 1, Index, [96, 12]]  # 5: P5 raw (96ch, /32)
  - [5, 1, Conv, [1024, 1, 1]]  # 6: P5 projected (1024ch, /32)

# YOLO12 head (adapted indices to match projected features: P3=2, P4=4, P5=6)
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 7: up P5
  - [[-1, 4], 1, Concat, [1]]  # 8: cat up P5 + P4
  - [-1, 2, A2C2f, [512, False, -1]]  # 9: refined P4
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 10: up refined P4
  - [[-1, 2], 1, Concat, [1]]  # 11: cat up P4 + P3
  - [-1, 2, A2C2f, [256, False, -1]]  # 12: refined P3
  - [-1, 1, Conv, [256, 3, 2]]  # 13: down P3 to P4
  - [[-1, 9], 1, Concat, [1]]  # 14: cat down P3 + refined P4
  - [-1, 2, A2C2f, [512, False, -1]]  # 15: refined P4
  - [-1, 1, Conv, [512, 3, 2]]  # 16: down P4 to P5
  - [[-1, 6], 1, Concat, [1]]  # 17: cat down P4 + P5
  - [-1, 2, C3k2, [1024, True]]  # 18: refined P5
  - [[12, 15, 18], 1, Detect, [nc]]  # Detect(P3, P4, P5)
"""

# Simpan YAML ke file
custom_yaml_path = Path("/content/yolo12n_mobilenetv3-small_panet.yaml")
with open(custom_yaml_path, "w") as f:
    f.write(custom_yaml_content)
print(f"✅ Custom YAML disimpan: {custom_yaml_path}")

# Inisialisasi model dengan custom YAML
model = YOLO(str(custom_yaml_path))

def train_model(model, data_yaml, project_name="train_icdas_mnetv3-small_panet", epochs=50):
    results = model.train(
        data=data_yaml,
        imgsz=640,
        epochs=epochs,
        batch=16,
        device=0,
        deterministic=True,
        project="runs/pure",
        name=project_name,
        pretrained=False  # Backbone pretrained via DEFAULT, head from scratch
    )
    return results

train_results = train_model(model, DATA_YAML)
print("✅ Training selesai! Model terbaik: runs/pure/train_icdas_mnetv3-small_panet/weights/best.pt")

loading Roboflow workspace...
loading Roboflow project...
✅ Dataset berhasil diunduh: /content/dental-caries-10/data.yaml
Struktur dataset: ['train', 'valid', 'README.dataset.txt', 'data.yaml', 'README.roboflow.txt', 'test']
✅ Custom YAML disimpan: /content/yolo12m_mobilenetv3-small_panet.yaml
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dental-caries-10/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1

KeyboardInterrupt: 